In [ ]:
from fastai.vision.all import *

In [ ]:
set_seed(2021)

In [ ]:
path = Path("../input/plant-pathology-2021-fgvc8")
path.ls()

In [ ]:
train_df = pd.read_csv(path/"train.csv")
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df["labels"].value_counts()

I use this [resized dataset](https://www.kaggle.com/ankursingh12/resized-plant2021) to save my time.  
Without this, it takes too much time to resizing images...😅

In [ ]:
item_tfms = [RandomResizedCrop(128, min_scale=0.75, ratio=(1., 1.))]
batch_tfms = [*aug_transforms(size=128, max_warp=0), Normalize.from_stats(*imagenet_stats)]

dls = ImageDataLoaders.from_df(
    df = train_df,
    folder = "../input/resized-plant2021/img_sz_512",
    item_tfms = item_tfms,
    batch_tfms = batch_tfms,
    splitter = RandomSplitter(valid_pct=0.1),
    label_delim = " ",
    bs=256
)

In [ ]:
# if you are a big fan of DataBlock API, you can use this code below

# db = DataBlock(
#     blocks=(ImageBlock, MultiCategoryBlock),
#     get_x=ColReader("image", pref="../input/resized-plant2021/img_sz_640/"),
#     get_y=ColReader("labels", label_delim=" "),
#     splitter=RandomSplitter(valid_pct=0.1),
#     item_tfms=item_tfms,
#     batch_tfms=batch_tfms
# )
# dls = db.dataloaders(train_df)

In [ ]:
dls.show_batch()

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints
# !cp ../input/resnet18/resnet18.pth /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
!cp ../input/resnet50/resnet50.pth /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth

In [ ]:
learn = cnn_learner(
    dls,
    resnet50,
    metrics=[accuracy_multi, F1ScoreMulti()]
).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(
    7,
    1e-1,
    cbs=[
        SaveModelCallback(),
        EarlyStoppingCallback(patience=3),
    ],
    freeze_epochs=3
)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
submission_df = pd.read_csv(path/"sample_submission.csv")
submission_df.head()

In [ ]:
test_image_path_series = submission_df["image"].apply(lambda x: f"../input/plant-pathology-2021-fgvc8/test_images/{x}")
test_image_path_series.head()

In [ ]:
test_dl = learn.dls.test_dl(test_image_path_series)
preds, _ = learn.tta(dl=test_dl)

In [ ]:
preds

In [ ]:
vocab = learn.dls.vocab
vocab

In [ ]:
threshold = 0.5

def pred_to_labels(pred):
    labels = []
    for i, probability in enumerate(pred):
        if probability > threshold:
            labels.append(vocab[i])
            
    return " ".join(labels)

In [ ]:
labels_list = [pred_to_labels(pred) for pred in preds]
labels_list

In [ ]:
submission_df["labels"] = labels_list
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv", index=False)